<p style="font-family: Arial; font-size:3em;color:purple; font-style:bold"><br>
Final Project - Machine Learning Short Term Stock<br><br> Prices with A2M.AX</p><br>
<br>
The aim of this project is to answer the following question:

**With the help of daily stock data and machine learning tools, are we able to predict short term future prices of stock?**

The reason for this investigation is that I've always been curious about whether future stock prices can be predicted with any accuracy whatsoever using historical time series data. 

The dataset used is from Yahoo Finance. It contains 1,180 data samples of daily stock data for the stock A2M.AX, listed on the ASX. The data range is from 31 March 2015 to 22 November 2019. 

You can obtain similar data yourself by creating a proile on Yahoo Finance, and adding whatever stock or stocks you're interested to your portfolio, then exporting the data using csv format.

Location: https://finance.yahoo.com/

In [57]:
%config IPCompleter.greedy=True

# Below can be used to change the width of the Jupyter notebook. I did this so that the full dataset with all variables can be seen without needing to 
# use the scrollbar

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

Let's load our data in to the notebook and check out a few key features of it to get an idea how it's shaped and how it looks.

In [58]:
data = pd.read_csv('A2M-ASX.csv')

In [66]:
print(data.shape)
[x for x in data.columns]

(1180, 26)


['date',
 'open',
 'high',
 'low',
 'close',
 'adj_close',
 'range',
 'range_change',
 'range_move',
 'price_change',
 'price_move',
 'volume',
 'qty_change',
 'qty_move',
 '5d_avg_range',
 'change_in_5d_avg_range',
 '5d_avg_range_move',
 '5d_avg_px',
 'change_in_5d_avg_px',
 '5d_avg_px_move',
 '5d_avg_qty',
 'change_in_5d_avg_qty',
 '5d_avg_qty_move',
 '10d_avg_px',
 'change_in_10d_avg_px',
 '10d_avg_px_move']

Off the bat we can see how many features each data sample has, and what these features are. As can be seen, it's a mixture of price and volume data for daily intervals.

Let's view some of the data.

In [67]:
data[:10]

,date,open,high,low,close,adj_close,range,range_change,range_move,price_change,...,5d_avg_range_move,5d_avg_px,change_in_5d_avg_px,5d_avg_px_move,5d_avg_qty,change_in_5d_avg_qty,5d_avg_qty_move,10d_avg_px,change_in_10d_avg_px,10d_avg_px_move
0,3/31/2015,0.555,0.595,0.530,0.565,0.565,0.065,NaN,Flat,0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/1/2015,0.575,0.580,0.555,0.565,0.565,0.025,-0.040,Down,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/2/2015,0.560,0.565,0.535,0.555,0.555,0.030,0.005,Up,-0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/7/2015,0.545,0.550,0.540,0.545,0.545,0.010,-0.020,Down,-0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/8/2015,0.545,0.545,0.530,0.540,0.540,0.015,0.005,Up,-0.005,...,NaN,0.554,NaN,NaN,2606643.8,NaN,NaN,NaN,NaN,NaN
5,4/9/2015,0.540,0.540,0.532,0.535,0.535,0.008,-0.007,Down,-0.005,...,Down,0.548,-0.006,Down,1784754.2,-821889.6,Down,NaN,NaN,NaN
6,4/10/2015,0.530,0.535,0.530,0.535,0.535,0.005,-0.003,Down,0.000,...,Down,0.542,-0.006,Down,944443.0,-840311.2,Down,NaN,NaN,NaN
7,4/13/2015,0.535,0.540,0.530,0.540,0.540,0.010,0.005,Up,0.005,...,Down,0.539,-0.003,Down,501676.6,-442766.4,Down,NaN,NaN,NaN
8,4/14/2015,0.540,0.540,0.535,0.540,0.540,0.005,-0.005,Down,0.000,...,Down,0.538,-0.001,Down,497519.4,-4157.2,Down,NaN,NaN,NaN
9,4/15/2015,0.540,0.550,0.540,0.540,0.540,0.010,0.005,Up,0.000,...,Down,0.538,0.000,Flat,548980.0,51460.6,Up,0.546,NaN,NaN


Immediately we can see there are a number of NaN values. These will need dealing with. 

Let's view them all first to see why they are there and if we should simply removing or altering to another value.

In [68]:
nulls = data.isnull().any(axis=1)
data[nulls]

,date,open,high,low,close,adj_close,range,range_change,range_move,price_change,...,5d_avg_range_move,5d_avg_px,change_in_5d_avg_px,5d_avg_px_move,5d_avg_qty,change_in_5d_avg_qty,5d_avg_qty_move,10d_avg_px,change_in_10d_avg_px,10d_avg_px_move
0,3/31/2015,0.555,0.595,0.530,0.565,0.565,0.065,NaN,Flat,0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4/1/2015,0.575,0.580,0.555,0.565,0.565,0.025,-0.040,Down,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/2/2015,0.560,0.565,0.535,0.555,0.555,0.030,0.005,Up,-0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4/7/2015,0.545,0.550,0.540,0.545,0.545,0.010,-0.020,Down,-0.010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4/8/2015,0.545,0.545,0.530,0.540,0.540,0.015,0.005,Up,-0.005,...,NaN,0.554,NaN,NaN,2606643.8,NaN,NaN,NaN,NaN,NaN
5,4/9/2015,0.540,0.540,0.532,0.535,0.535,0.008,-0.007,Down,-0.005,...,Down,0.548,-0.006,Down,1784754.2,-821889.6,Down,NaN,NaN,NaN
6,4/10/2015,0.530,0.535,0.530,0.535,0.535,0.005,-0.003,Down,0.000,...,Down,0.542,-0.006,Down,944443.0,-840311.2,Down,NaN,NaN,NaN
7,4/13/2015,0.535,0.540,0.530,0.540,0.540,0.010,0.005,Up,0.005,...,Down,0.539,-0.003,Down,501676.6,-442766.4,Down,NaN,NaN,NaN
8,4/14/2015,0.540,0.540,0.535,0.540,0.540,0.005,-0.005,Down,0.000,...,Down,0.538,-0.001,Down,497519.4,-4157.2,Down,NaN,NaN,NaN
9,4/15/2015,0.540,0.550,0.540,0.540,0.540,0.010,0.005,Up,0.000,...,Down,0.538,0.000,Flat,548980.0,51460.6,Up,0.546,NaN,NaN


We see that the null data is only our data which contains averages for preceding days, as these data points don't have enough previous days to average.

Let's remove these data points as they shouldn't affect our end results too much, and there's no other way to know what these data points should be without digging for more data.

In [69]:
data = data.dropna()
print(data.shape)
data.head()

(1170, 26)


,date,open,high,low,close,adj_close,range,range_change,range_move,price_change,...,5d_avg_range_move,5d_avg_px,change_in_5d_avg_px,5d_avg_px_move,5d_avg_qty,change_in_5d_avg_qty,5d_avg_qty_move,10d_avg_px,change_in_10d_avg_px,10d_avg_px_move
10,4/16/2015,0.545,0.545,0.525,0.53,0.53,0.020,0.010,Up,-0.01,...,Up,0.537,-0.001,Down,526133.4,-22846.6,Down,0.5425,-0.0035,Down
11,4/17/2015,0.525,0.540,0.520,0.54,0.54,0.020,0.000,Flat,0.01,...,Up,0.538,0.001,Up,600670.6,74537.2,Up,0.5400,-0.0025,Down
12,4/20/2015,0.535,0.535,0.520,0.52,0.52,0.015,-0.005,Down,-0.02,...,Up,0.534,-0.004,Down,562333.0,-38337.6,Down,0.5365,-0.0035,Down
13,4/21/2015,0.525,0.530,0.520,0.52,0.52,0.010,-0.005,Down,0.00,...,Up,0.530,-0.004,Down,618294.0,55961.0,Up,0.5340,-0.0025,Down
14,4/22/2015,0.520,0.530,0.520,0.52,0.52,0.010,0.000,Flat,0.00,...,Flat,0.526,-0.004,Down,511622.2,-106671.8,Down,0.5320,-0.0020,Down


Now that our data is clean, lets's find some more interesting aspects of our data. 

Using Panda's `.describe()` method we can analyse basic statistical properties of the data set. Note this only works for numerical data.

Given the length of the data, and the change of A2M.AX over time, this is not very useful for basic price data, but is more interesting for range and volume data, as well as averaged price data.

In [70]:
data.describe()

,open,high,low,close,adj_close,range,range_change,price_change,volume,qty_change,5d_avg_range,change_in_5d_avg_range,5d_avg_px,change_in_5d_avg_px,5d_avg_qty,change_in_5d_avg_qty,10d_avg_px,change_in_10d_avg_px
count,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1170.000000,1.170000e+03,1170.000000,1.170000e+03,1.170000e+03,1170.000000,1170.000000,1170.000000,1170.000000,1.170000e+03,1.170000e+03,1170.000000,1170.000000
mean,6.322292,6.418415,6.228620,6.320603,6.320603,0.189796,1.752137e-04,0.011496,4.851845e+06,4.765803e+03,0.189169,0.000383,6.297850,0.011002,4.839798e+06,6.276960e+03,6.272018,0.010297
std,5.064178,5.133915,4.997526,5.062496,5.062496,0.187427,1.421932e-01,0.217534,5.401034e+06,4.971388e+06,0.159886,0.035292,5.054732,0.098545,4.053856e+06,1.280930e+06,5.048993,0.070383
min,0.470000,0.475000,0.455000,0.465000,0.465000,0.000000,-1.420000e+00,-2.110000,0.000000e+00,-4.136640e+07,0.008600,-0.369000,0.471000,-0.508000,2.654982e+05,-1.135242e+07,0.473000,-0.268000
25%,1.740000,1.760500,1.715000,1.735000,1.735000,0.045000,-4.000000e-02,-0.050000,2.226296e+06,-1.185753e+06,0.049700,-0.010000,1.734000,-0.016000,2.653897e+06,-3.211968e+05,1.721875,-0.010000
50%,4.230000,4.350000,4.192500,4.305000,4.305000,0.130000,-3.330670e-16,0.000000,3.506864e+06,-1.133290e+05,0.136000,0.000000,4.180000,0.004000,3.752113e+06,-1.185940e+04,4.037000,0.004000
75%,10.940000,11.162500,10.807500,10.970000,10.970000,0.290000,4.000000e-02,0.060000,5.370262e+06,1.030598e+06,0.305500,0.010000,10.982000,0.038000,5.862383e+06,3.051626e+05,10.926750,0.030000
max,17.200001,17.299999,17.030001,17.129999,17.129999,2.085000,1.205000e+00,2.590000,6.139730e+07,4.167741e+07,1.062000,0.393000,17.010000,0.726000,3.343366e+07,9.973982e+06,16.778000,0.397000


Let's fix some of the rounding as it doesn't need to be 6 d.p. for all features.